In [1]:
import openai
import sentence_transformers
import evadb
from tqdm import tqdm
cursor = evadb.connect(evadb_dir=("./eva_db")).cursor()

In [2]:
cursor.query(""" CREATE OR REPLACE FUNCTION EmbedText IMPL './sentence_feature_extractor.py' """).df()
cursor.query(""" SELECT EmbedText('hello world') """).df()
cursor.query(""" SELECT Similarity(EmbedText('hello world'), EmbedText('hello there world')) """).df()

/home/oscar/.cache/pypoetry/virtualenvs/hfsm-0EdPmVNd-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


,distance
0,0.147227


In [3]:
cursor.query(""" DROP TABLE IF EXISTS courses """).df()
cursor.query(""" CREATE TABLE courses ( number TEXT)""").df()
cursor.query(""" SELECT * FROM courses """).df()


""


In [4]:
# Add the course numbers from ./cnumbers.txt
from pathlib import Path
course_numbers = Path("./cnumbers.txt").read_text().split("\n")[:-1]
for i, course_number in enumerate(course_numbers):
    if i % 5 != 0 and course_number!='6422' and course_number!='7643':
        continue
    cursor.query(""" INSERT INTO courses (number) VALUES ('{}') """.format(course_number)).df()
cursor.query(""" SELECT * FROM courses """).df()
# 8s for 400

,_row_id,number
0,1,1100
1,2,1331
2,3,1802
3,4,2051
4,5,2335
...,...,...
62,63,7643
63,64,7645
64,65,7650
65,66,7742


In [10]:
# fetch urls from bing
# register the bing_search function at ./bing_search.py
cursor.query(""" CREATE OR REPLACE FUNCTION BingSearch IMPL './bing_search.py' """).df()
cursor.query(""" CREATE OR REPLACE FUNCTION CONCAT IMPL './concat.py' """).df()

# cursor.query("""SELECT BingSearch('hello world')""").df()
# gatech CS "7641" course homepage -site:omscs.gatech.edu -"omscs"
numbers_links = cursor.query(""" SELECT number, BingSearch(CONCAT(CONCAT('gatech CS "', number), '" course homepage -site:omscs.gatech.edu -site:oscar.gatech.edu -site:catalog.gatech.edu -"omscs" 2023')) FROM courses """).df()
numbers_links
# 6s for 20

,number,links
0,1100,"[https://www.scs.gatech.edu/, https://gt-stude..."
1,1331,"[https://cs1331.gitlab.io/syllabus.html, https..."
2,1802,"[https://www.scs.gatech.edu/, https://www.cc.g..."
3,2051,[https://gt-student-wiki.org/mediawiki/index.p...
4,2335,"[https://www.scs.gatech.edu/, https://www.cc.g..."
...,...,...
62,7643,[https://sites.cc.gatech.edu/classes/AY2023/cs...
63,7645,"[https://www.scs.gatech.edu/, https://www.cc.g..."
64,7650,[https://sites.cc.gatech.edu/classes/AY2021/cs...
65,7742,"[https://www.scs.gatech.edu/, https://www.cc.g..."


In [11]:
numbers_links.iloc[1]['links']

['https://cs1331.gitlab.io/syllabus.html',
 'https://faculty.cc.gatech.edu/~stasko/1331/',
 'https://classes.cc.gatech.edu/AY2021/cs1331a_spring/index.html',
 'https://gt-student-wiki.org/mediawiki/index.php/CS_1331',
 'https://faculty.cc.gatech.edu/~stasko/1331/course.html',
 'https://admission.gatech.edu/dual-enrollment/distance-computer-science']

In [22]:
# create a table relating (number, link)
cursor.query(""" DROP TABLE IF EXISTS course_links """).df()
cursor.query(""" CREATE TABLE course_links ( number TEXT, link TEXT)""").df()
# copy the course numbers from courses table by selecting, iterating, and inserting
for i, row in numbers_links.iterrows():
    ct = 0
    for j, link in enumerate(row[1]):
        # insist that cc.gatech.edu is in the link
        if "cc.gatech.edu" not in link:
            continue
        # exclue #https://prod-cc.cc.gatech.edu/ms-computer-science-specializations
        if 'ms-computer-science-specializations' in link:
            continue
        # exclude pdfs
        if '.pdf' in link or 'program-study' in link or 'www.cc.gatech.edu' in link:
            continue
        # exclude degree-programs
        if 'degree-programs' in link:
            continue
        ct += 1
        if ct > 2:
            continue
        cursor.query("""
        INSERT INTO course_links (number, link)
        VALUES ('{}', '{}')
        """.format(row[0], link)).df()
number_link = cursor.query("""SELECT * FROM course_links""").df()
number_link
# 3s for 20

/tmp/ipykernel_71938/2220012522.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for j, link in enumerate(row[1]):
/tmp/ipykernel_71938/2220012522.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  """.format(row[0], link)).df()


,_row_id,number,link
0,1,1331,https://faculty.cc.gatech.edu/~stasko/1331/
1,2,1331,https://classes.cc.gatech.edu/AY2021/cs1331a_s...
2,3,3220,https://faculty.cc.gatech.edu/~milos/Teaching/...
3,4,3630,https://faculty.cc.gatech.edu/~seth/Teaching/c...
4,5,3630,https://faculty.cc.gatech.edu/~seth/Teaching/c...
5,6,4210,https://sites.cc.gatech.edu/fac/rama/
6,7,4270,https://sites.cc.gatech.edu/classes/AY2013/cs4...
7,8,4460,https://sites.cc.gatech.edu/home/stasko/4460/
8,9,4476,https://faculty.cc.gatech.edu/~hays/compvision/
9,10,4476,https://faculty.cc.gatech.edu/~hays/compvision...


In [23]:
cursor.query(""" SELECT * FROM course_links WHERE number='6422' """).df()

,_row_id,number,link
0,15,6422,https://faculty.cc.gatech.edu/~jarulraj/course...


In [24]:
cursor.query(""" CREATE OR REPLACE FUNCTION DownloadURL IMPL './download_url.py' """).df()
cursor.query(""" CREATE OR REPLACE FUNCTION ExtractText IMPL './extract_soup.py' """).df()
cursor.query(""" CREATE OR REPLACE FUNCTION ChunkText   IMPL './context_chunker.py' """).df()
# fetch html from links
number_link_chunks = cursor.query("""SELECT number, link, ChunkText(ExtractText(DownloadURL(link))) FROM course_links""").df()
number_link_chunks
# 9s

,number,link,chunks
0,1331,https://faculty.cc.gatech.edu/~stasko/1331/,"[CS1331 Fall 2022, Section C Home | Schedule |..."
1,1331,https://classes.cc.gatech.edu/AY2021/cs1331a_s...,"[CS1331 Spring 2021, Sections A & GR Home | Sc..."
2,3220,https://faculty.cc.gatech.edu/~milos/Teaching/...,[CS 3220 - Processor Design CS3220 - Processor...
3,3630,https://faculty.cc.gatech.edu/~seth/Teaching/c...,[CS 3630: Introduction to Robotics and Percept...
4,3630,https://faculty.cc.gatech.edu/~seth/Teaching/c...,[CS 3630: Introduction to Robotics and Percept...
5,4210,https://sites.cc.gatech.edu/fac/rama/,[kishore.html Umakishore Ramachandran Professo...
6,4270,https://sites.cc.gatech.edu/classes/AY2013/cs4...,[CS4270-CS8803DCL Spring 2012 (College of Comp...
7,4460,https://sites.cc.gatech.edu/home/stasko/4460/,[CS 4460 - Intro. to Information Visualization...
8,4476,https://faculty.cc.gatech.edu/~hays/compvision/,[Computer Vision CS 4476-A Computer Vision Fal...
9,4476,https://faculty.cc.gatech.edu/~hays/compvision...,[CS 4476 Computer Vision CS 4476-B Computer Vi...


In [25]:
# create a table of (number, link, chunk)
cursor.query(""" DROP TABLE IF EXISTS course_chunks """).df()
cursor.query(""" CREATE TABLE course_chunks ( number TEXT, link TEXT, chunk TEXT ) """).df()
# copy the course numbers from courses table by selecting, iterating, and inserting
for i, row in tqdm(number_link_chunks.iterrows()):
    for chunk in row[2]:
        # remove quotes marks from chunk
        chunk = chunk.replace("'", "").replace('"', '')
        cursor.query("""
        INSERT INTO course_chunks (number, link, chunk)
        VALUES ('{}', '{}', '{}')
        """.format(row[0], row[1], chunk)).df()
number_link_chunk = cursor.query("""SELECT * FROM course_chunks""").df()
number_link_chunk
# 30s (x20 = 10m)
# 3m for 20%


0it [00:00, ?it/s]/tmp/ipykernel_71938/3193276649.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for chunk in row[2]:
/tmp/ipykernel_71938/3193276649.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  """.format(row[0], row[1], chunk)).df()
35it [00:38,  1.11s/it]


,_row_id,number,link,chunk
0,1,1331,https://faculty.cc.gatech.edu/~stasko/1331/,"CS1331 Fall 2022, Section C Home | Schedule | ..."
1,2,1331,https://faculty.cc.gatech.edu/~stasko/1331/,- Introduction to Object-Oriented Programming ...
2,3,1331,https://faculty.cc.gatech.edu/~stasko/1331/,object-oriented programming such as encapsulat...
3,4,1331,https://faculty.cc.gatech.edu/~stasko/1331/,programming skills. Course Objectives Students...
4,5,1331,https://faculty.cc.gatech.edu/~stasko/1331/,construction of Java programs. Demonstrate pro...
...,...,...,...,...
1398,1399,7650,https://sites.cc.gatech.edu/classes/AY2021/cs7...,students may benefit from your questions and o...
1399,1400,7650,https://sites.cc.gatech.edu/classes/AY2021/cs7...,"(not exhaustive!) Introduction to NLP, CMU Alg..."
1400,1401,7650,https://sites.cc.gatech.edu/classes/AY2021/cs7...,Berkeley Natural Language Processing with Deep...
1401,1402,7650,https://sites.cc.gatech.edu/classes/AY2021/cs7...,"Speech and Language Processing, 3rd edition (E..."


In [28]:
# create table context_prompts number | prof_ctx | title_ctx | desc_ctx | languages_ctx | grade_ctx
cursor.query(""" DROP TABLE IF EXISTS title_context_prompts """).df()
cursor.query(""" CREATE TABLE title_context_prompts ( number TEXT, title_ctx TEXT ) """).df()
# print course numbers present in chunks
numbers = cursor.query(""" SELECT number FROM course_chunks """).df()
numbers = numbers.drop_duplicates()
# iterate thru numbers and build the context_prompts table
for i, row in tqdm(numbers.iterrows()):
    number = row[0]
    # title
    title_ctx_retrieval_prompt = f'CS {number}. Course Title. Introduction to High Level Computing. Advanced Database Systems. Operating Systems. Machine Learning.'
    best_ctxs = cursor.query(f""" SELECT chunk FROM course_chunks WHERE number='{number}' ORDER BY Similarity(EmbedText(chunk), EmbedText('{title_ctx_retrieval_prompt}')) LIMIT 4 """).df()
    title_ctx = '\n\n'.join(best_ctxs['chunk'].tolist())
    cursor.query(f""" INSERT INTO title_context_prompts (number, title_ctx) VALUES ('{number}', '{title_ctx}') """).df()
#2m for 20 -> 40m for 400

0it [00:00, ?it/s]/tmp/ipykernel_71938/297561898.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  number = row[0]
/home/oscar/.cache/pypoetry/virtualenvs/hfsm-0EdPmVNd-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
1it [00:00,  1.00it/s]/tmp/ipykernel_71938/297561898.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a valu

In [29]:
# register gpt function
cursor.query(""" CREATE OR REPLACE FUNCTION GPT IMPL './gpt.py' """).df()
# get professor names
title_question_prompt = "Based on the fragments figure out the title of the course. Output only the title of the course not the course number. If it cannot be determined or guessed invent a reasonable title."
course_titles = cursor.query(f""" SELECT number, GPT(CONCAT(CONCAT('{title_question_prompt}', title_ctx), '\n\nANSWER:')) FROM title_context_prompts """).df()
# cursor.query(f" SELECT GPT('what is the name of \n\nthe \nprofessor?') ").df()
# 3s for short answers.
course_titles

,number,reply
0,1331,Introduction to Java Programming and Data Stru...
1,3220,Processor Design
2,3630,CS 3630: Introduction to Robotics
3,4210,Introduction to Computer Systems and Networking
4,4270,Hands-on Networking Technologies
5,4460,Intro. to Information Visualization
6,4476,Computer Vision and Image Processing
7,4675,Database Systems
8,6230,High Performance Computing: Tools and Applicat...
9,6422,Introduction to Database Systems


In [30]:
# iterate thru the course_titles and create a table of ctx_prompts for the other fields
# create a new table of context_prompts number | prof_ctx | title_ctx | desc_ctx | languages_ctx | grade_ctx
cursor.query(""" DROP TABLE IF EXISTS context_prompts """).df()
cursor.query(""" CREATE TABLE context_prompts ( number TEXT, title TEXT, prof_ctx TEXT, desc_ctx TEXT, languages_ctx TEXT, grade_ctx TEXT ) """).df()
# iter thru course_titles
for i, row in tqdm(course_titles.iterrows()):
    number = row[0]
    title = row[1]
    # title
    title = title
    # prof
    prof_ctx_retrieval_prompt = f'Professor / Instructor '
    best_ctxs = cursor.query(f""" SELECT chunk FROM course_chunks WHERE number='{number}' ORDER BY Similarity(EmbedText(chunk), EmbedText('{prof_ctx_retrieval_prompt}')) LIMIT 4 """).df()
    prof_ctx = '\n\n'.join(best_ctxs['chunk'].tolist())
    # desc
    desc_ctx_retrieval_prompt = f'{title}. Key Topics. Techniques. Skills.'
    best_ctxs = cursor.query(f""" SELECT chunk FROM course_chunks WHERE number='{number}' ORDER BY Similarity(EmbedText(chunk), EmbedText('{desc_ctx_retrieval_prompt}')) LIMIT 4 """).df()
    desc_ctx = '\n\n'.join(best_ctxs['chunk'].tolist())
    # languages
    languages_ctx_retrieval_prompt = f'Programming Languages. Python. C++. Javascript. CUDA.'
    best_ctxs = cursor.query(f""" SELECT chunk FROM course_chunks WHERE number='{number}' ORDER BY Similarity(EmbedText(chunk), EmbedText('{languages_ctx_retrieval_prompt}')) LIMIT 4 """).df()
    languages_ctx = '\n\n'.join(best_ctxs['chunk'].tolist())
    # grade
    grade_ctx_retrieval_prompt = f'Class grade percentage %. Exam / Participation / Project / Assignment.'
    best_ctxs = cursor.query(f""" SELECT chunk FROM course_chunks WHERE number='{number}' ORDER BY Similarity(EmbedText(chunk), EmbedText('{grade_ctx_retrieval_prompt}')) LIMIT 4 """).df()
    grade_ctx = '\n\n'.join(best_ctxs['chunk'].tolist())
    # insert into table
    cursor.query(f""" INSERT INTO context_prompts (number, title, prof_ctx, desc_ctx, languages_ctx, grade_ctx) VALUES ('{number}', '{title}', '{prof_ctx}', '{desc_ctx}', '{languages_ctx}', '{grade_ctx}') """).df()
# show
context_prompts = cursor.query(""" SELECT * FROM context_prompts """).df()
context_prompts
# 30s for 20 -> 10m for 400


0it [00:00, ?it/s]/tmp/ipykernel_71938/1082996899.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  number = row[0]
/tmp/ipykernel_71938/1082996899.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title = row[1]
/home/oscar/.cache/pypoetry/virtualenvs/hfsm-0EdPmVNd-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx

,_row_id,number,title,prof_ctx,desc_ctx,languages_ctx,grade_ctx
0,1,1331,Introduction to Java Programming and Data Stru...,"CS1331 Fall 2022, Section C Home | Schedule | ...",Required Textbook Title: Introduction to Java ...,user interface (GUI) programs. Prerequisites A...,during regular class hours or the final exam s...
1,2,3220,Processor Design,OSCAR). You can still see your final exam and ...,run on this processor. In addition to learning...,(hardware functionality) and code generation (...,on the report. Project assignment 2 10% Grade ...
2,3,3630,CS 3630: Introduction to Robotics,Course Information | Lecture Schedule | Resour...,CS 3630: Introduction to Robotics and Percepti...,ASAP. All programming assignments will be comp...,will each be worth 6% of the final course grad...
3,4,4210,Introduction to Computer Systems and Networking,a Professor in the College of Computing. Recen...,Introduction to Computer Systems and Networkin...,kishore.html Umakishore Ramachandran Professor...,"coverage of material, etc.). Also from CoC sch..."
4,5,4270,Hands-on Networking Technologies,Office hours: By appointment Email: constantin...,networking course. We will also cover a number...,"(tcpdump, netstat, ping, traceroute). Addition...",students. the midterm exam will be on October ...
5,6,4460,Intro. to Information Visualization,"talk or whisper to others in class, or otherwi...",wide variety of information visualization tech...,"interactive design exercises. Additionally, mu...","based on attendance/pop quizzes, homework assi..."
6,7,4476,Computer Vision and Image Processing,handin Contact Info and Office Hours: If possi...,previous knowledge of visual computing or sign...,"MacOS, Windows, and Linux. The course does not...",your final grade. There may be an extra credit...
7,8,4675,Database Systems,the assigned date or time? A: You should infor...,"selection of database courses, both at undergr...",core computer science concepts such as data st...,technology review (20%). Class participation i...
8,9,6230,High Performance Computing: Tools and Applicat...,but expected. In addition to lectures and henc...,Introduction to High Performance Computing for...,assignments from this course will be written i...,for instructor and TA). Course Requirements & ...
9,10,6422,Introduction to Database Systems,The programming assignments are geared towards...,Objectives This is the first part of a two-par...,(CS 3210) (recommended) Introduction to Databa...,course will be tentatively based on the follow...


In [31]:
cursor.query(""" CREATE OR REPLACE FUNCTION GPT IMPL './gpt.py' """).df()
# prompts
prof_question_prompt = "Based on the fragments figure out the name of the professor or instructor. \
Output only the professor name and no other text. Format your answer as `[First] [Last]`. Output `unknown` if the professor does not appear in the fragments. """
desc_question_prompt = "Create a three sentence concise summary of the course based on the fragments. Do not use long lists."
languages_question_prompt = "Based on the fragments figure out the programming languages used in the course. Output a list of languages separated by commas. Output `unknown` if the languages do not appear in the fragments."
grade_question_prompt = "Based on the fragments figure out the grade percentage breakdown of the course. Output a list of percentages separated by commas. Output `unknown` if the percentages do not appear in the fragments."
# professors
print('professors...')
query = f""" SELECT GPT(CONCAT(CONCAT('{prof_question_prompt}', prof_ctx), '\n\nANSWER:')) FROM context_prompts"""
professors = cursor.query(query).df()
# descriptions
print('description...')
query = f""" SELECT GPT(CONCAT(CONCAT('{desc_question_prompt}', desc_ctx), '\n\nANSWER:')) FROM context_prompts"""
descriptions = cursor.query(query).df()
# languages
print('languages...')
query = f""" SELECT GPT(CONCAT(CONCAT('{languages_question_prompt}', languages_ctx), '\n\nANSWER:')) FROM context_prompts"""
languages = cursor.query(query).df()
# grade
print('grade...')
query = f""" SELECT GPT(CONCAT(CONCAT('{grade_question_prompt}', grade_ctx), '\n\nANSWER:')) FROM context_prompts"""
grades = cursor.query(query).df()


professors...
description...
languages...
grade...


In [32]:
# assemble into course_fields table
# create table course_fields
cursor.query(""" DROP TABLE IF EXISTS course_fields """).df()
cursor.query(""" CREATE TABLE course_fields ( number TEXT, title TEXT, professor TEXT, description TEXT, languages TEXT, grade TEXT ) """).df()
# zip together the fields and iter and insert
for row in tqdm(zip(context_prompts['number'], context_prompts['title'], professors['reply'], descriptions['reply'], languages['reply'], grades['reply'])):
    cursor.query(f""" INSERT INTO course_fields (number, title, professor, description, languages, grade) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}') """).df()
# show
cursor.query(""" SELECT * FROM course_fields """).df()


23it [00:00, 33.35it/s]


,_row_id,number,title,professor,description,languages,grade
0,1,1331,Introduction to Java Programming and Data Stru...,John,"This course, CS1331, covers the fundamentals o...",Java,unknown
1,2,3220,Processor Design,unknown,CS 3220 is an intermediate-level course that f...,unknown,"10, 15, 5, 15, 20"
2,3,3630,CS 3630: Introduction to Robotics,Seth Hutchinson,CS 3630: Introduction to Robotics and Percepti...,"Python, unknown","10, 6, unknown"
3,4,4210,Introduction to Computer Systems and Networking,unknown,The course is an introduction to computer syst...,unknown,unknown
4,5,4270,Hands-on Networking Technologies,unknown,The course is a networking course that aims to...,"tcpdump, netstat, ping, traceroute",unknown
5,6,4460,Intro. to Information Visualization,unknown,"This course, ""4460 - Intro. to Information Vis...",unknown,unknown
6,7,4476,Computer Vision and Image Processing,unknown,This course is an introduction to computer vis...,"Python, PyTorch","12, 16, 16, 16, 16, unknown"
7,8,4675,Database Systems,unknown,This course is an introduction to database sys...,"programming languages, unknown","20%, 15%, 15%, 50%"
8,9,6230,High Performance Computing: Tools and Applicat...,Edmond Chow,"The course ""Introduction to High Performance C...","C, C++, CUDA, unknown","60, 40"
9,10,6422,Introduction to Database Systems,Joy Arulraj,This course is the first part of a two-part se...,"Java, C++","55, 15, 30"


In [33]:
# dalle-3 image generation from title and description
# register dalle function
cursor.query(""" CREATE OR REPLACE FUNCTION DALLE IMPL './dalle.py' """).df()
dalle_prompt = "Create a course website header image for the following course: " # not used
full_course_table = cursor.query(""" SELECT number, title, professor, description, languages, grade, DALLE(CONCAT(title, '. ', description)) FROM course_fields """).df()
print('Done generating images. Copying...')
# copy full_course_table into an sql table
cursor.query(""" DROP TABLE IF EXISTS full_course_table """).df()
cursor.query(""" CREATE TABLE full_course_table ( number TEXT, title TEXT, professor TEXT, description TEXT, languages TEXT, grade TEXT, img_url TEXT ) """).df()
for i, row in tqdm(full_course_table.iterrows()):
    cursor.query(f""" INSERT INTO full_course_table (number, title, professor, description, languages, grade, img_url) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}') """).df()
# show
cursor.query(""" SELECT * FROM full_course_table """).df()
# DALLE IS RATE LIMITED TO 5req/min SO 300req/hr
# 3reqs/s so 40m for 110 courses

sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
sleeping 12s
Done generating images. Copying...


0it [00:00, ?it/s]/tmp/ipykernel_71938/3003524564.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cursor.query(f""" INSERT INTO full_course_table (number, title, professor, description, languages, grade, img_url) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}') """).df()
23it [00:00, 28.92it/s]


,_row_id,number,title,professor,description,languages,grade,img_url
0,1,1331,Introduction to Java Programming and Data Stru...,John,"This course, CS1331, covers the fundamentals o...",Java,unknown,https://oaidalleapiprodscus.blob.core.windows....
1,2,3220,Processor Design,unknown,CS 3220 is an intermediate-level course that f...,unknown,"10, 15, 5, 15, 20",https://oaidalleapiprodscus.blob.core.windows....
2,3,3630,CS 3630: Introduction to Robotics,Seth Hutchinson,CS 3630: Introduction to Robotics and Percepti...,"Python, unknown","10, 6, unknown",https://oaidalleapiprodscus.blob.core.windows....
3,4,4210,Introduction to Computer Systems and Networking,unknown,The course is an introduction to computer syst...,unknown,unknown,https://oaidalleapiprodscus.blob.core.windows....
4,5,4270,Hands-on Networking Technologies,unknown,The course is a networking course that aims to...,"tcpdump, netstat, ping, traceroute",unknown,https://oaidalleapiprodscus.blob.core.windows....
5,6,4460,Intro. to Information Visualization,unknown,"This course, ""4460 - Intro. to Information Vis...",unknown,unknown,https://oaidalleapiprodscus.blob.core.windows....
6,7,4476,Computer Vision and Image Processing,unknown,This course is an introduction to computer vis...,"Python, PyTorch","12, 16, 16, 16, 16, unknown",https://oaidalleapiprodscus.blob.core.windows....
7,8,4675,Database Systems,unknown,This course is an introduction to database sys...,"programming languages, unknown","20%, 15%, 15%, 50%",https://oaidalleapiprodscus.blob.core.windows....
8,9,6230,High Performance Computing: Tools and Applicat...,Edmond Chow,"The course ""Introduction to High Performance C...","C, C++, CUDA, unknown","60, 40",https://oaidalleapiprodscus.blob.core.windows....
9,10,6422,Introduction to Database Systems,Joy Arulraj,This course is the first part of a two-part se...,"Java, C++","55, 15, 30",https://oaidalleapiprodscus.blob.core.windows....


In [34]:
# create vector index
cursor.query(""" DROP INDEX IF EXISTS course_vector_idx """).df()
cursor.query("""
CREATE INDEX course_vector_idx
ON full_course_table (EmbedText(description))
USING FAISS
""").df()

11-25-2023 19:12:17 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_index:0116] Index course_vector_idx does not exist, therefore cannot be dropped.


/home/oscar/.cache/pypoetry/virtualenvs/hfsm-0EdPmVNd-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


,0
0,Index course_vector_idx successfully added to ...


In [35]:
# test the vector idx
query = 'databases'
cursor.query(f"""
SELECT * FROM full_course_table ORDER BY Similarity(EmbedText(description), EmbedText('{query}'))
""").df()

/home/oscar/.cache/pypoetry/virtualenvs/hfsm-0EdPmVNd-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


,_row_id,number,title,professor,description,languages,grade,img_url
0,16,6754,Database Systems,unknown,Georgia Tech offers a variety of database cour...,"Introduction to Database Systems, Database Ana...",unknown,https://oaidalleapiprodscus.blob.core.windows....
1,8,4675,Database Systems,unknown,This course is an introduction to database sys...,"programming languages, unknown","20%, 15%, 15%, 50%",https://oaidalleapiprodscus.blob.core.windows....
2,10,6422,Introduction to Database Systems,Joy Arulraj,This course is the first part of a two-part se...,"Java, C++","55, 15, 30",https://oaidalleapiprodscus.blob.core.windows....
3,15,6730,Data Visualization,unknown,This course focuses on the fundamental princip...,unknown,unknown,https://oaidalleapiprodscus.blob.core.windows....
4,18,7450,Information Visualization,unknown,This course is an advanced information visuali...,"Vega-Lite, D3, JavaScript","15, 15, 20, 10, 40",https://oaidalleapiprodscus.blob.core.windows....
5,1,1331,Introduction to Java Programming and Data Stru...,John,"This course, CS1331, covers the fundamentals o...",Java,unknown,https://oaidalleapiprodscus.blob.core.windows....
6,6,4460,Intro. to Information Visualization,unknown,"This course, ""4460 - Intro. to Information Vis...",unknown,unknown,https://oaidalleapiprodscus.blob.core.windows....
7,14,6675,Advanced Internet Application Development,unknown,The course is a graduate level class on Advanc...,"Java, C, CGI","10, 25, 50, 15",https://oaidalleapiprodscus.blob.core.windows....
8,2,3220,Processor Design,unknown,CS 3220 is an intermediate-level course that f...,unknown,"10, 15, 5, 15, 20",https://oaidalleapiprodscus.blob.core.windows....
9,12,6474,Deep Learning,unknown,This course on Deep Learning covers a wide ran...,"Python, PyTorch","64%, 36%, 1%",https://oaidalleapiprodscus.blob.core.windows....


In [36]:
# serve over flask
from flask import Flask, request, jsonify
app = Flask(__name__)

# allow cross origin requests from any domain
from flask_cors import CORS
CORS(app)

@app.route('/get_similar_courses', methods=['GET'])
def get_similar_games():
    query = request.args.get('query')
    out = cursor.query(f"""
    SELECT * FROM full_course_table ORDER BY Similarity(EmbedText('{query}'), EmbedText(description));
    """).df()
    out_recs = out.to_dict(orient="records")[:15]
    # replace every url with https with https: (BC EVADB) doesn't let me insert w/ ?
    return jsonify(out_recs)

app.run(host="localhost", port=5010)

# this can be accessed as follows: http://localhost:5000/get_similar_courses?query=fast-paced%20action%20game

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5010
Press CTRL+C to quit
/home/oscar/.cache/pypoetry/virtualenvs/hfsm-0EdPmVNd-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
127.0.0.1 - - [25/Nov/2023 19:12:38] "GET /get_similar_courses?query=undefined HTTP/1.1" 200 -
/home/oscar/.cache/pypoetry/virtualenvs/hfsm-0EdPmVNd-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] 